# EDA (Exploratory Data Analysis)

Utilizarei o dataset fictício que gerei no notebook [DATASET_CREATION.ipynb](DATASET_CREATION.ipynb).

Por se tratar de um dataset fictício e randômico, os dados não possuem nenhuma relação entre sí, logo os dados e gráficos não terão nenhum sentido ao comparar com um exemplo real.

In [1]:
import pandas as pd
import plotly.express as px
# importando os módulos para este notebook

## 1. Conhecendo o Dataset

In [2]:
df = pd.read_csv('../data.csv', parse_dates=['date'])
# lendo o datset fazendo o parse da coluna de data, para ler no
# tipo de dado correto (datetime)

In [3]:
df

,date,invested,returned,sales_channel
0,2020-01-01,74025.0,108610.0,Print Advertising
1,2020-01-02,33993.0,137278.0,tv
2,2020-01-03,92217.0,28635.0,Social Media
3,2020-01-04,79190.0,34487.0,Radio
4,2020-01-05,40134.0,43405.0,Social Media
...,...,...,...,...
1141,2022-11-07,-346.0,138362.0,Radio
1142,2020-12-24,76492.0,123577.0,tv
1143,2021-01-02,34097.0,127769.0,Social Media
1144,2022-11-24,68731.0,51069.0,Social Media


In [4]:
df.info()
# verificando informações gerais do df (dataframe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1146 entries, 0 to 1145
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1146 non-null   datetime64[ns]
 1   invested       1128 non-null   float64       
 2   returned       1097 non-null   float64       
 3   sales_channel  1063 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 35.9+ KB


### 1.1 Tratando Valores Duplicados

In [5]:
df.duplicated().sum()
# verificado presença de valore duplicados

50

In [6]:
df = df.drop_duplicates()
# removendo valores duplicados por serem em pouca quantidade
# quando comparado com o tamanho do df

In [7]:
df.duplicated().sum()
# operação realizada com sucesso!

0

### 1.2 Tratando Valores Nulos

In [8]:
df.isnull().sum()
# verificando presença de valores nulos

date              0
invested         18
returned         47
sales_channel    80
dtype: int64

In [9]:
df = df.dropna()
# removendo os valores nulos por serem em pouca quantidade
# comparado com todo o df

In [10]:
df.isnull().sum()

date             0
invested         0
returned         0
sales_channel    0
dtype: int64

### 1.3 Tratando Valores Inconsistêntes (*typos* ou erros de digitação/input)

In [11]:
df['sales_channel'].value_counts()
# verificado inconsistência nos valores desta coluna

Print Advertising    170
social media         167
Radio                162
Social Media         160
TV                   154
tv                   143
Name: sales_channel, dtype: int64

In [12]:
df['sales_channel'] = df['sales_channel'].str.capitalize()
# padronizando os valores para remover as inconsistências

In [13]:
df['sales_channel'].value_counts()
# tudo ok agora!

Social media         327
Tv                   297
Print advertising    170
Radio                162
Name: sales_channel, dtype: int64

### 1.4 Tratando Valores Inconsistêntes (valores monetários errôneos)

In [14]:
df.describe()
# verificando presença de valores inconsistêntes nas colunas monetárias

,invested,returned
count,956.000000,956.000000
mean,58958.731172,73444.398536
std,25578.066344,39219.049224
min,-4896.000000,-4624.000000
25%,38945.250000,40327.000000
50%,58991.500000,74791.000000
75%,81851.500000,106973.000000
max,99997.000000,139917.000000


In [15]:
df.query('invested < 1 or returned < 1').head()
# verificando o problema de forma mais individual

,date,invested,returned,sales_channel
7,2020-01-08,66152.0,-1347.0,Social media
56,2020-02-26,92236.0,-1853.0,Social media
58,2020-02-28,-4143.0,56365.0,Social media
70,2020-03-11,85854.0,-326.0,Radio
77,2020-03-18,-1598.0,41756.0,Tv


In [16]:
len(df.query('invested < 1 or returned < 1'))
# verificando quantos registros seguem esse erro

54

In [17]:
df = df.drop(df.query('invested < 1 or returned < 1').index).reset_index(drop=True)
# removendo estes erros por serem insignificantes em comparação com o df ao todo

In [18]:
df.query('invested < 1 or returned < 1')
# removido!

,date,invested,returned,sales_channel


## 2. Gerando Novas Features (Feature Engineering)

Esta etapa ajudará na visualização e descoberta de padrões no nosso dataframe, e ajudará principalmente no treinamento do modelo de *machine learning*

In [19]:
df
# como o df está por enquanto...

,date,invested,returned,sales_channel
0,2020-01-01,74025.0,108610.0,Print advertising
1,2020-01-02,33993.0,137278.0,Tv
2,2020-01-03,92217.0,28635.0,Social media
3,2020-01-04,79190.0,34487.0,Radio
4,2020-01-05,40134.0,43405.0,Social media
...,...,...,...,...
897,2022-12-27,34306.0,14006.0,Tv
898,2022-12-28,42139.0,37726.0,Print advertising
899,2022-12-29,87553.0,47059.0,Radio
900,2022-12-30,20543.0,13637.0,Tv


In [20]:
df['roi'] = ((df['returned'] - df['invested']) / df['invested']) * 100
df
# adicionando a coluna do ROI de acordo com o calculo apropriado e multiplicando por 100 para receber a porcentagem

,date,invested,returned,sales_channel,roi
0,2020-01-01,74025.0,108610.0,Print advertising,46.720702
1,2020-01-02,33993.0,137278.0,Tv,303.841967
2,2020-01-03,92217.0,28635.0,Social media,-68.948242
3,2020-01-04,79190.0,34487.0,Radio,-56.450309
4,2020-01-05,40134.0,43405.0,Social media,8.150197
...,...,...,...,...,...
897,2022-12-27,34306.0,14006.0,Tv,-59.173322
898,2022-12-28,42139.0,37726.0,Print advertising,-10.472484
899,2022-12-29,87553.0,47059.0,Radio,-46.250842
900,2022-12-30,20543.0,13637.0,Tv,-33.617291


In [21]:
df['month'] = df['date'].dt.month_name()
df['day_of_week'] = df['date'].dt.day_name()
df.head()
# definindo colunas para nome do mês e dia da semana

,date,invested,returned,sales_channel,roi,month,day_of_week
0,2020-01-01,74025.0,108610.0,Print advertising,46.720702,January,Wednesday
1,2020-01-02,33993.0,137278.0,Tv,303.841967,January,Thursday
2,2020-01-03,92217.0,28635.0,Social media,-68.948242,January,Friday
3,2020-01-04,79190.0,34487.0,Radio,-56.450309,January,Saturday
4,2020-01-05,40134.0,43405.0,Social media,8.150197,January,Sunday


## 3. Análise Estatística

In [22]:
df.describe().round()
# verificando dados estatísticos para as colunas numéricas
# e arredondando os valores decimais para uma melhor visualização

,invested,returned,roi
count,902.0,902.0,902.0
mean,60732.0,75301.0,51.0
std,23634.0,37788.0,113.0
min,20047.0,10154.0,-89.0
25%,40787.0,43063.0,-31.0
50%,59914.0,75735.0,26.0
75%,82430.0,107867.0,98.0
max,99997.0,139917.0,560.0


In [23]:
px.line(df, x='date', y=['invested', 'returned'])
# observando a série temporal dos valores investidos e retornados

# por conta de nossa base de dados randômica, a visualização acaba não
# sendo muito efetiva por não condizer com condições normais da realidade

In [24]:
px.histogram(df, x='sales_channel', color='month')
# observando a distribuição dos canais de venda com uma
# dimensionalidade extra baseada nos meses

# novamente podemos ver que, por conta dos dados terem sido gerados artificialmente,
# não há muita diferença entre os canais de venda e principalmente entre os meses

In [25]:
px.box(df, y=['invested', 'returned'])
# visualizando a distribuição dos valores investidos e retornados

In [26]:
px.box(df['roi'])
# visualizando a distribuição do ROI onde pudemos detectar outliers em seu topo (bolinhas azuis)
# outliers são valores que estão muito afastados dos valores comuns mais agregados entre sí (valores atípicos)

In [27]:
px.histogram(df, x='roi')
# verificando a distribuição do ROI através de um histograma

In [28]:
px.scatter(df, x='invested', y='returned', color='month')
# visualizando a correlação entre os valores investidos e retornados
# com a dimensionalidade extra baseada nos meses

# aqui é possível ver a perfeita ramdomicidade dos dados que geramos,
# porém em um caso real, poderiamos ver aqui algum tipo de relação